### Our first goal is to import raw data from Matlab files and coerce to df's

In [2]:
## Imports
from scipy import io
import pandas as pd
import numpy as np
import os

In [3]:
## Load Matlab files
file_biomass = r'C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\raw\biomass_abund.mat'
mat_biomass = io.loadmat(file_biomass)
file_landings = r'C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\raw\landings.mat'
mat_landings = io.loadmat(file_landings)
file_ecosystem = r'C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\raw\metadata_ecosystem.mat'
mat_ecosystem = io.loadmat(file_ecosystem)

In [4]:
## Coerce mat_biomass to df_biomass
remove_keys = ['__header__', '__version__', '__globals__'] # download data irrelevant to data analysis
for key in remove_keys:
    mat_biomass.pop(key) # remove these keys from the imported Matlab dictionary

for key in list(mat_biomass.keys()):
    if len(mat_biomass[key])==1:
        mat_biomass[key] = mat_biomass[key][0] # reassign ndarray as value, dropping dtype information

df_biomass = pd.DataFrame.from_dict(mat_biomass, orient= 'index').T

In [5]:
## Coerce mat_landings to df_landings
remove_keys = ['__header__', '__version__', '__globals__'] # download data irrelevant to data analysis
for key in remove_keys:
    mat_landings.pop(key) # remove these keys from the imported MATLAB dictionary


for key in list(mat_landings.keys()):
    if len(mat_landings[key])==1:
        mat_landings[key] = mat_landings[key][0] # reassign ndarray as value, dropping dtype information

df_landings = pd.DataFrame.from_dict(mat_landings, orient= 'index').T

In [6]:
## Coerce mat_ecosystem to df_ecosystem
remove_keys = ['__header__', '__version__', '__globals__'] # download data irrelevant to data analysis
for key in remove_keys:
    mat_ecosystem.pop(key) # remove these keys from the imported MATLAB dictionary

df_ecosystem = pd.DataFrame.from_dict(mat_ecosystem, orient= 'index').T

### Our second goal is to load and coerce AMO text data

In [7]:
## Load AMO data, to df with read_table
# Adapted from: https://stackoverflow.com/questions/48063620/pandas-read-csv-for-multiple-delimiters
file_AMO = r'C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\raw\amon.sm.long.data'
df_AMO = pd.read_table(file_AMO, delimiter='\s+|  |    ', skiprows=1, engine='python')
df_AMO.columns = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [8]:
# Slice df to rows with real year value, and complete data for 12 months
dec_check = df_AMO['Dec'] > -99 # Only rows with float type, and w/in expected value
year_check = df_AMO['Year'].str.len() == 4 # Only rows where Year column is a 4 digit year string
df_AMO = df_AMO[dec_check & year_check]

### Our third goal is to load global ocean temperature anomally data

In [9]:
# List for all .asc files in timeseries folder, filter for only annual ocean data
# Adapted from: https://www.geeksforgeeks.org/python-list-files-in-a-directory/
temperature_folder = r"C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\raw\timeseries" 
dir_list = pd.Series(os.listdir(temperature_folder))
ocean_files = [file for file in dir_list if 'ann.ocean.' in file]
print(ocean_files)

['aravg.ann.ocean.00N.30N.v4.0.1.201711.asc', 'aravg.ann.ocean.00N.90N.v4.0.1.201711.asc', 'aravg.ann.ocean.20N.90N.v4.0.1.201711.asc', 'aravg.ann.ocean.20S.20N.v4.0.1.201711.asc', 'aravg.ann.ocean.30N.60N.v4.0.1.201711.asc', 'aravg.ann.ocean.30S.00N.v4.0.1.201711.asc', 'aravg.ann.ocean.60N.90N.v4.0.1.201711.asc', 'aravg.ann.ocean.60S.30S.v4.0.1.201711.asc', 'aravg.ann.ocean.60S.60N.v4.0.1.201711.asc', 'aravg.ann.ocean.90S.00N.v4.0.1.201711.asc', 'aravg.ann.ocean.90S.20S.v4.0.1.201711.asc', 'aravg.ann.ocean.90S.60S.v4.0.1.201711.asc', 'aravg.ann.ocean.90S.90N.v4.0.1.201711.asc']


These are all timeseries files for annual ocean temperature anomalies. Landings in our fishery metadata, I have searched for latitude of each:

Long_Island_Sound   41.0748° N

Georges_Bank    40.8155° N

Gulf_of_Maine   43.1336° N

Mid_Atlantic_Bight  39° 25' 1.2" N

Southern_New_England   40° N

https://www.fisheries.noaa.gov/new-england-mid-atlantic/commercial-fishing/southern-new-england-exemption-area

Chesapeake_Bay  37.5214° N

Narragansett_Bay    41.6220° N

Hudson_River (estuary)  40°42' N

Connecticut_Shoreline   41.2700° N

Delaware_Bay    39.1202° N

Ambrose_Channel 40.488215° N

It could be considered to subset data for the latitudes above, however this is global data over the latitude range. Oceans other than the Atlantic are in all dataset, so the decision made is to select global temperature anomalies for full range of latitude (90°S to 90°N)

In [10]:
ocean_file = [file for file in ocean_files if '90S.90N' in file] # select file for full latitude range

df_ocean_temp = pd.read_table(f'{temperature_folder}\{ocean_file[0]}', delimiter='\s+|   |    ', skiprows=0, engine='python')
# Adapted from: https://datascientyst.com/reset-column-names-index-pandas/
df_ocean_temp = df_ocean_temp.T.reset_index().T # move first row of data from column names into first row of df
df_ocean_temp[0] = df_ocean_temp[0].astype('int')
df_ocean_temp.columns = ['year', 'anomaly of temperature (K)', 'total error variance (K**2)', \
                    'high-frequency error variance (K**2)', 'low-frequency error variance (K**2)', \
                    'bias error variance (K**2)']

In [11]:
df_biomass

,ecosystem,species,year,biomass,biomass_SE,abundance,abundance_SE,biomass_index,biomass_index_SE,abundance_index,abundance_index_units,abundance_index_SE,avg_len,avg_mass,source,agency,season
0,CB,American_eel_age-1+,1988,NaN,NaN,NaN,NaN,NaN,NaN,1.26,no./5min_tow,NaN,0.3055,NaN,VIMS_Juvenile_Trawl,VIMS,Apr-Jun
1,CB,American_eel_age-1+,1989,NaN,NaN,NaN,NaN,NaN,NaN,7.93,no./5min_tow,NaN,0.2921,NaN,VIMS_Juvenile_Trawl,VIMS,Apr-Jun
2,CB,American_eel_age-1+,1990,NaN,NaN,NaN,NaN,NaN,NaN,4.85,no./5min_tow,NaN,0.2701,NaN,VIMS_Juvenile_Trawl,VIMS,Apr-Jun
3,CB,American_eel_age-1+,1991,NaN,NaN,NaN,NaN,NaN,NaN,2.06,no./5min_tow,NaN,0.2616,NaN,VIMS_Juvenile_Trawl,VIMS,Apr-Jun
4,CB,American_eel_age-1+,1992,NaN,NaN,NaN,NaN,NaN,NaN,8.29,no./5min_tow,NaN,0.2516,NaN,VIMS_Juvenile_Trawl,VIMS,Apr-Jun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,SNE,Yellowtail_flounder,2003,0.046378,0.004109,NaN,NaN,NaN,NaN,NaN,nd,NaN,NaN,NaN,NEFSC_Bottom_Trawl_Survey,NMFS/NEFSC,nd
26133,SNE,Yellowtail_flounder,2004,0.028945,0.003155,NaN,NaN,NaN,NaN,NaN,nd,NaN,NaN,NaN,NEFSC_Bottom_Trawl_Survey,NMFS/NEFSC,nd
26134,SNE,Yellowtail_flounder,2005,0.021592,0.004607,NaN,NaN,NaN,NaN,NaN,nd,NaN,NaN,NaN,NEFSC_Bottom_Trawl_Survey,NMFS/NEFSC,nd
26135,SNE,Yellowtail_flounder,2006,0.04994,0.007284,NaN,NaN,NaN,NaN,NaN,nd,NaN,NaN,NaN,NEFSC_Bottom_Trawl_Survey,NMFS/NEFSC,nd


In [12]:
df_landings

,ecosystem,species,year,landings_biomass,landings_abund,landings_biomass_index,landings_abund_index,avg_len,avg_mass,source,agency,season
0,GB,Atlantic_herring,1993,0.430273,NaN,NaN,NaN,NaN,NaN,CFDBS,NMFS/NEFSC,nd
1,GB,Atlantic_herring,1994,0.297367,NaN,NaN,NaN,NaN,NaN,CFDBS,NMFS/NEFSC,nd
2,GB,Atlantic_herring,1995,0.328735,NaN,NaN,NaN,NaN,NaN,CFDBS,NMFS/NEFSC,nd
3,GB,Atlantic_herring,1996,0.781348,NaN,NaN,NaN,NaN,NaN,CFDBS,NMFS/NEFSC,nd
4,GB,Atlantic_herring,1997,0.398372,NaN,NaN,NaN,NaN,NaN,CFDBS,NMFS/NEFSC,nd
...,...,...,...,...,...,...,...,...,...,...,...,...
4880,CB,Skates_NS,2003,NaN,NaN,62.6,NaN,NaN,NaN,nd,nd,nd
4881,CB,Skates_NS,2005,NaN,NaN,38.7,NaN,NaN,NaN,nd,nd,nd
4882,CB,Skates_NS,2006,NaN,NaN,20.4,NaN,NaN,NaN,nd,nd,nd
4883,CB,Skates_NS,2007,NaN,NaN,68.4,NaN,NaN,NaN,nd,nd,nd


In [13]:
df_ecosystem

,site_code,full_name,area,mean_depth,type
0,LIS,Long_Island_Sound,LIS_from_New_London_to_Greenwich_CT,5-46_m,Estuary
1,GB,Georges_Bank,42154.44,81.9,Ocean
2,GoM,Gulf_of_Maine,76482.74,167.3,Ocean
3,MAB,Mid_Atlantic_Bight,79661.9,nd,Ocean
4,SNE,Southern_New_England,28004.29,nd,Ocean
5,CB,Chesapeake_Bay,nd,nd,Estuary
6,NB,Narragansett_Bay,420,nd,Estuary
7,HR,Hudson_River,nd,nd,Estuary
8,CTS,Connecticut_Shoreline,nd,nd,Estuary
9,Del,Delaware_Bay,nd,nd,Estuary


In [14]:
df_AMO

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
4,1861,0.055,0.055,0.055,0.055,0.055,0.056,0.056,0.057,0.056,0.055,0.056,0.057
5,1862,0.058,0.058,0.062,0.065,0.066,0.067,0.067,0.066,0.066,0.066,0.068,0.068
6,1863,0.071,0.074,0.078,0.081,0.081,0.082,0.086,0.089,0.091,0.089,0.087,0.086
7,1864,0.084,0.086,0.087,0.089,0.090,0.091,0.094,0.096,0.097,0.096,0.094,0.093
8,1865,0.091,0.090,0.090,0.089,0.089,0.087,0.084,0.083,0.084,0.085,0.086,0.087
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2012,0.143,0.143,0.143,0.144,0.145,0.147,0.148,0.150,0.152,0.155,0.157,0.158
156,2013,0.159,0.159,0.159,0.158,0.158,0.156,0.154,0.154,0.153,0.153,0.151,0.151
157,2014,0.150,0.151,0.154,0.156,0.158,0.160,0.162,0.163,0.163,0.164,0.164,0.164
158,2015,0.165,0.167,0.169,0.169,0.168,0.166,0.166,0.166,0.164,0.163,0.161,0.161


In [15]:
df_ocean_temp

,year,anomaly of temperature (K),total error variance (K**2),high-frequency error variance (K**2),low-frequency error variance (K**2),bias error variance (K**2)
index,1880,-0.261522,0.003293,0.000073,0.000850,0.002371
0,1881,-0.197709,0.003328,0.000073,0.000877,0.002378
1,1882,-0.207949,0.003306,0.000073,0.000897,0.002336
2,1883,-0.250376,0.003284,0.000073,0.00091,0.002302
3,1884,-0.327371,0.003231,0.000073,0.000922,0.002236
...,...,...,...,...,...,...
132,2013,0.327718,0.006762,0.000025,0.000002,0.006735
133,2014,0.421128,0.006762,0.000025,0.000002,0.006735
134,2015,0.523553,0.006762,0.000025,0.000002,0.006735
135,2016,0.541199,0.006762,0.000025,0.000002,0.006735


### Data cleaning will occur in the next notebook, for now write to CSV.

In [17]:
csv_folder = r"C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\interim"
df_biomass.to_csv(fr'{csv_folder}\biomass_data.csv')
df_landings.to_csv(fr'{csv_folder}\landings_data.csv')
df_ecosystem.to_csv(fr'{csv_folder}\ecosystem_data.csv')
df_AMO.to_csv(fr'{csv_folder}\AMO_data.csv')
df_ocean_temp.to_csv(fr'{csv_folder}\ocean_temp_data.csv')